In [33]:
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras import datasets, layers, models
from PIL import Image, ImageChops, ImageEnhance, ImageOps
from os import listdir, mkdir
from os.path import isfile, join
from sklearn import svm, model_selection, datasets
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
%matplotlib notebook
from tensorflow.keras.models import load_model
import matplotlib.image as mpimg
import imageio as im
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from time import time


In [34]:
# dimensión de las imágenes
img_width, img_height = 100,100

#ruta de las imágenes
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/test'

nb_train_samples = 2000
nb_validation_samples = 800

#Se puede disminuir las épocas para una prueba más rápida
#Podría demorar 1 minuto por época aproximadamente
epochs = 30
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [35]:
# ESTO ES PARA SALVAR LAS IMÁGENES CON LAS NUEVAS DIMENSIONES
#ruta = "dataset/train"
#newdir = "resizeImages"
#mkdir(newdir)

#def ls(ruta):
#    return [arch for arch in listdir(ruta) if isfile(join(ruta, arch))]

#def resized(origen, destino,ancho,largo):
    
#    for name in ls(origen):

        #print (name)
        #imagen = Image.open(origen+"/"+name)
        #new_image = imagen.resize((ancho, largo))
        #new_image.save(destino+"/"+"resized_"+name)

In [36]:
#resized(ruta, newdir, img_width, img_height)

In [37]:
#ESTRUCTURA INICIAL, parámetros temporales

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)       

In [38]:
#Compilación

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics =['accuracy'])

#Creación del DataSET a partir de las imágenes en la carpeta dataset

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#REESCALADO

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Conjunto de datos y etiquetas para ENTRENAMIENTO
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Conjunto de datos y etiquetas para VALIDACIÓN
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 4035 images belonging to 2 classes.
Found 996 images belonging to 2 classes.


In [ ]:
epochs_to_wait_for_improve = 2

early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                        patience=epochs_to_wait_for_improve)

checkpoint_callback = ModelCheckpoint('early.h5', 
                                      monitor='val_loss', 
                                      verbose=1, 
                                      save_best_only=True, 
                                      mode='min')


#Entrenamiento
start_time = time()
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[early_stopping_callback, checkpoint_callback])

# Salvado de pesos como archivo
model.save_weights('estructura_1.h5')

elapsed_time = time() - start_time

Epoch 1/30
30/31 [============================>.] - ETA: 14s - loss: 0.6498 - accuracy: 0.6125
Epoch 00001: val_loss improved from inf to 0.50173, saving model to early.h5
31/31 [==============================] - 450s 15s/step - loss: 0.6479 - accuracy: 0.6169 - val_loss: 0.5017 - val_accuracy: 0.8164
Epoch 2/30
30/31 [============================>.] - ETA: 1s - loss: 0.3979 - accuracy: 0.8451
Epoch 00002: val_loss improved from 0.50173 to 0.40323, saving model to early.h5
31/31 [==============================] - 65s 2s/step - loss: 0.3977 - accuracy: 0.8461 - val_loss: 0.4032 - val_accuracy: 0.8255
Epoch 3/30
30/31 [============================>.] - ETA: 2s - loss: 0.3236 - accuracy: 0.8766
Epoch 00003: val_loss improved from 0.40323 to 0.26982, saving model to early.h5
31/31 [==============================] - 76s 2s/step - loss: 0.3235 - accuracy: 0.8765 - val_loss: 0.2698 - val_accuracy: 0.9167
Epoch 4/30
30/31 [============================>.] - ETA: 1s - loss: 0.3005 - accuracy: 0.

In [ ]:
print(elapsed_time)

In [ ]:
#Test

holdout_gen = ImageDataGenerator(rescale = 1/255)
holdout = holdout_gen.flow_from_directory(train_data_dir,
                                    shuffle=False,
                                    batch_size=50,
                                    class_mode='binary')

#model_eval = model.predict_generator(holdout, use_multiprocessing=True)

#preds = pd.DataFrame(model_eval, columns = holdout.class_indices.keys())
#preds['filename'] = holdout.filenames
#preds['truth'] = preds['filename'].apply(os.path.dirname)
#preds['predicted_class'] = preds[list(holdout.class_indices.keys())].idxmax(1)
#preds.head()


#print(str(np.mean(preds['predicted_class'] == preds['truth']) * 100) + "% Accuracy")

In [ ]:
#Curva aciertos en épocas 
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

evaluation = model.evaluate_generator(train_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=2)


In [ ]:
# Plot ENTRENAMIENTO & VALIDACI accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot ENTRENAMIENTO & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
img_path = 'dataset/test/freshapples/rotated_by_15_Screen Shot 2018-06-08 at 4.59.49 PM.png'

img = image.load_img(img_path, target_size=(100, 100))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

plt.imshow(img_tensor[0])
plt.show()

print(img_tensor.shape)

In [ ]:
image = np.expand_dims(image, axis=0) #para agregar una dimensión extra

In [ ]:
layer_outputs = [layer.output for layer in model.layers[:5]] # Extrae los resultados de las 5 capas superiores.
activation_model = models.Model(inputs=model.input, outputs=layer_outputs) # Crea un modelo que devolverá las salidas, dada la entrada del modelo

In [ ]:
activations = activation_model.predict(img_tensor) # Devuelve una lista de cinco matrices

In [ ]:
layer_names = [model]
for layer in model.layers[:12]:
    layer_names.append(layer.name) # nombre de las capas
    
images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations): # Muestra los mapas de características
    n_features = layer_activation.shape[-1] # Número de características en el mapa de características
    size = layer_activation.shape[1]
    n_cols = n_features // images_per_row # Mosaicos de los canales de activación en esta matriz
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): #Coloca cada filtro en una cuadrícula horizontal
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-procesa la función para que sea visualmente agradable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
def get_truths(df, class_label):
    y_truth = df['truth'] == class_label
    return y_truth.astype(int).values, df[class_label].values